In [ ]:
import numpy as np
import pandas as pd
import nltk
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import re
import random 
import collections
# use multiprocessing for multicore ops
import copy

In [ ]:
data2 = pd.read_csv("../data/questions.csv", delimiter=',', encoding="utf-8-sig")
data1 = copy.deepcopy(data2[:50000])
print(data1.shape)
print(data1.iloc[0:1,3:])
#random.shuffle(data1)
data1.dropna(inplace=True)

In [ ]:
X = data1[["question1","question2"]]
y = data1[['is_duplicate']]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
#nltk.download()
from nltk.corpus import stopwords # Import the stop word list
print(stopwords.words("english")) 


In [ ]:
def question_to_words( raw_quest, lower=True, remove_not = False ):
    
    lemmetizer = nltk.WordNetLemmatizer()
    
    letters_only = re.sub("[^a-zA-Z]", " ", raw_quest)
    if lower==True:
        letters_only = letters_only.lower()
    words = nltk.word_tokenize(letters_only)
    #words = nltk.word_tokenize(s)
    stops = set(stopwords.words("english")) 
    not_stop = ['no', 'nor', 'not', "don't", "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
    if remove_not == True:
        #stop = stops.difference(not_stop)
        stops = ['the','a','an','and','but','if','or','because','as','what','which','this','that','these','those','then',
              'just','so','than','such','both','through','about','for','is','of','while','during','to','What','Which',
              'Is','If','While','This']
    meaningful_words = [w for w in words if not w in stops]
    
    meaningful_words = [lemmetizer.lemmatize(w) for w in meaningful_words]
    return( " ".join( meaningful_words ))

In [ ]:
#X_train['question2']= X_train['question2'].apply(question_to_words)
#X_train = X_train.apply(question_to_words)
print(X_train.shape)

#a = X_train[['question1']]
#a = a.apply(question_to_words)
X_train['question1'] = X_train['question1'].apply(question_to_words)
X_train['question2'] = X_train['question2'].apply(question_to_words)
#b=X_train[['question2']]
#b.apply(question_to_words)
X_train


In [ ]:
print(X_train.shape)
print(X_train.head)

In [ ]:
print("Creating the bag of words...\n")
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 
#add whitelisting
# use ngrams , ngram_range=(1,2) check for various combinations

In [ ]:
#corpus = X_train["question1"] + " " + X_train["question2"]
corpus = X_train["question1"] + " " + X_train["question2"]
cv = vectorizer.fit(corpus)

cv
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
print(X_train)
print(y_train)

In [ ]:
#new_data = X_train + y_train
new_data = pd.concat([X_train, y_train], axis=1, join_axes=[X_train.index])
print(new_data.head)


In [ ]:
duplicates = new_data[new_data.is_duplicate==1]
nonduplicates = new_data[new_data.is_duplicate==0]
print(duplicates.shape)
print(nonduplicates.shape)

In [ ]:
duplicates.head

In [ ]:
def getSimilarity(df):
    r,c = df.shape
    print(str(r)+ str(c))
    final_vec = []
    for j in range(r):
        a = cv.transform([df.iloc[j].question1])
        b = cv.transform([df.iloc[j].question2])
        sim = round(cosine_similarity(a,b).ravel()[0], 3)
        c = df.iloc[j].is_duplicate
        final_vec.append([sim, c])
    return final_vec


In [ ]:
sim = getSimilarity(new_data)

In [ ]:
sim

In [ ]:
df1 = pd.DataFrame(sim)
df1.columns = ["cos_sim", "y"]
df1


In [ ]:
X = df1['cos_sim']
y = df1['y']


In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf= RandomForestClassifier()
X = X.values.reshape(1, -1)
y = y.values.reshape(1, -1)
clf.fit(X,y)
